![Neptune + PyTorch Ignite](https://neptune.ai/wp-content/uploads/2023/09/ignite.svg)

# Neptune + PyTorch Ignite

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/pytorch-ignite/notebooks/Neptune_PyTorch_Ignite.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/pytorch-ignite/notebooks/Neptune_PyTorch_Ignite.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/neptune-ai/org/pytorch-ignite-integration/e/PYTOR-30/charts"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a>


<a target="_blank" href="https://docs.neptune.ai/integrations/ignite/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

This guide will show you how to:

* Create a `NeptuneLogger()`,
* Log training metrics to Neptune using `NeptuneLogger()`,
* Upload model checkpoints to Neptune using `NeptuneSaver()`.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -U neptune pytorch-ignite scikit-plot torchvision

**Note**: If running on Google Colab, restart the kernel and continue execution from the next cell to avoid a `ContextualVersionConflict` error.

This error is caused by Colab coming with `future==0.16.0` preinstalled, while `torchvision` updates this to a newer version.

## Import libraries

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Normalize, ToTensor

from ignite.engine import create_supervised_evaluator, create_supervised_trainer, Events
from ignite.metrics import Accuracy, Loss
from ignite.utils import setup_logger

## Define hyper-parameters

In [ ]:
params = {
    "train_batch_size": 64,
    "val_batch_size": 64,
    "epochs": 10,
    "lr": 0.1,
    "momentum": 0.5,
}

## Create model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

In [ ]:
model = Net()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)  # Move model before creating optimizer

## Define DataLoader()

In [ ]:
def get_data_loaders(train_batch_size, val_batch_size):
    data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

    train_loader = DataLoader(
        MNIST(download=True, root=".", transform=data_transform, train=True),
        batch_size=train_batch_size,
        shuffle=True,
    )

    val_loader = DataLoader(
        MNIST(download=False, root=".", transform=data_transform, train=False),
        batch_size=val_batch_size,
        shuffle=False,
    )
    return train_loader, val_loader

In [ ]:
train_loader, val_loader = get_data_loaders(params["train_batch_size"], params["val_batch_size"])

## Create optimizer, trainer, and logger

In [ ]:
optimizer = SGD(model.parameters(), lr=params["lr"], momentum=params["momentum"])
criterion = nn.CrossEntropyLoss()

trainer = create_supervised_trainer(model, optimizer, criterion, device=device)

## (Neptune) Create NeptuneLogger()

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass
from ignite.contrib.handlers.neptune_logger import NeptuneLogger

neptune_logger = NeptuneLogger(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
import neptune
from ignite.contrib.handlers.neptune_logger import NeptuneLogger

neptune_logger = NeptuneLogger(
    api_token=neptune.ANONYMOUS_API_TOKEN,
    project="common/pytorch-ignite-integration",
)

**To open the run, click the Neptune link that appears in the console output.**

This will be updated live once training starts.

## (Neptune) Attach logger to the trainer

In [ ]:
trainer.logger = setup_logger("Trainer")

neptune_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED(every=100),
    tag="training",
    output_transform=lambda loss: {"batchloss": loss},
)

## Create evaluators

In [ ]:
metrics = {"accuracy": Accuracy(), "loss": Loss(criterion)}

train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)

validation_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED)
def compute_metrics(engine):
    train_evaluator.run(train_loader)
    validation_evaluator.run(val_loader)

## (Neptune) Attach logger to training and validation evaluators

In [ ]:
from ignite.contrib.handlers.neptune_logger import global_step_from_engine

In [ ]:
train_evaluator.logger = setup_logger("Train Evaluator")

neptune_logger.attach_output_handler(
    train_evaluator,
    event_name=Events.EPOCH_COMPLETED,  # logging at the end of each epoch
    tag="training",
    metric_names="all",
    global_step_transform=global_step_from_engine(
        trainer
    ),  # takes the epoch of the trainer instead of train_evaluator
)

In [ ]:
validation_evaluator.logger = setup_logger("Validation Evaluator")

neptune_logger.attach_output_handler(
    validation_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="validation",
    metric_names="all",
    global_step_transform=global_step_from_engine(
        trainer
    ),  # takes the epoch of the trainer instead of train_evaluator
)

## (Neptune) Log optimizer parameters

In [ ]:
neptune_logger.attach_opt_params_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED(every=100),
    optimizer=optimizer,
)

## (Neptune) Log model's normalized weights and gradients after each iteration

In [ ]:
from ignite.contrib.handlers.neptune_logger import WeightsScalarHandler

neptune_logger.attach(
    trainer,
    log_handler=WeightsScalarHandler(model, reduction=torch.norm),
    event_name=Events.ITERATION_COMPLETED(every=100),
)

In [ ]:
from ignite.contrib.handlers.neptune_logger import GradsScalarHandler

neptune_logger.attach(
    trainer,
    log_handler=GradsScalarHandler(model, reduction=torch.norm),
    event_name=Events.ITERATION_COMPLETED(every=100),
)

## (Neptune) Save model checkpoints
__Note:__ `NeptuneSaver` currently does not work on Windows

In [ ]:
from ignite.handlers import Checkpoint
from ignite.contrib.handlers.neptune_logger import NeptuneSaver


def score_function(engine):
    return engine.state.metrics["accuracy"]


to_save = {"model": model}

handler = Checkpoint(
    to_save=to_save,
    save_handler=NeptuneSaver(neptune_logger),
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="validation_accuracy",
    global_step_transform=global_step_from_engine(trainer),
)

# validation_evaluator.add_event_handler(Events.COMPLETED, handler) # Uncomment to save model checkpoints on MacOS/Linux

## Run trainer

In [ ]:
trainer.run(train_loader, max_epochs=params["epochs"])

Head back to the run on Neptune to watch it being updated live!

## (Neptune) Logging additional metadata after training
You can access the Neptune run through the `.experiment` attribute of the `NeptuneLogger` object.

### (Neptune) Log hyper-parameters

In [ ]:
neptune_logger.experiment["params"] = params

### (Neptune) Upload trained model

In [ ]:
torch.save(model.state_dict(), "model.pth")
neptune_logger.experiment["trained_model"].upload("model.pth")

## (Neptune) Stop logging

In [ ]:
neptune_logger.close()

## Analyze logged metadata in the Neptune app

Go to the run link and explore metadata (metrics, params, model checkpoints) that were logged to the run in Neptune.